<a href="https://colab.research.google.com/github/gden173/stat4402_tute_paper/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Necessary Imports to Run Pytorch
import torch   
import torchvision
from torch import nn
import  torch.nn.functional as F
from torch import optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms

In [3]:
transform = transforms.ToTensor()
train_data = MNIST("/data", download=True, train=True, transform=transform)
test_data = MNIST("/data",download=True, train=False, transform=transform)

# Batch Size of the Training data
batch_size = 128

# Data Loaders
train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data = DataLoader(test_data)

Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [26]:
class CNN(nn.Module):  
  def __init__(self):   
    super(CNN, self).__init__() 
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)    
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)    
    self.fc1 =  nn.Linear(32 * (13 * 13), 100)
    self.fc2  = nn.Linear(100, 10)  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.pool(x)).view( -1, 32 * (13 * 13))
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [50]:
def score(model, test):
  model.eval()
  num = len(test)
  correct = 0
  with torch.no_grad():
    for _, (x,y) in enumerate(test):
      x = x.cuda()
      y = y.cuda()
      out = model(x)
      correct += torch.sum(out.max(1)[1] == y).item()
  correct /= num  
  model.train()
  return correct

In [53]:
def train(model,data,test,  num_epochs=100, batch_size = 128, learning_rate = 3 * 10 **(-3)):
  """
  """
  model.cuda()
  model.train()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  for epoch in range(num_epochs):
    for batch_idx, (x, y) in enumerate(data):
      x = x.cuda()
      y = y.cuda()
      model.zero_grad()
      out = model(x)
      loss = criterion(out,y)
      loss.backward()
      optimizer.step()
    print("Epoch:{}  Accuracy:{}".format(epoch, score(model,test)))
    



In [54]:
net = CNN()
train(net,train_data,test_data,  num_epochs=30)

Epoch:0  Accuracy:0.9771
Epoch:1  Accuracy:0.9787
Epoch:2  Accuracy:0.9838
Epoch:3  Accuracy:0.9859
Epoch:4  Accuracy:0.9843
Epoch:5  Accuracy:0.9859
Epoch:6  Accuracy:0.9859
Epoch:7  Accuracy:0.9822
Epoch:8  Accuracy:0.9854
Epoch:9  Accuracy:0.9864
Epoch:10  Accuracy:0.9844


KeyboardInterrupt: ignored

In [43]:
net(train_data.dataset[0][0].reshape(1,1,28,28).cuda()).max(1)[1]

tensor([5], device='cuda:0')

In [ ]:
def train():
  pass

In [ ]:
def score():
  pass